In [1]:
"""
Created on Sun Oct 25 2018
@author: Kimin Lee
"""
from __future__ import print_function
import numpy as np
import os
import lib_regression
from sklearn.metrics import roc_auc_score
# import argparse

from sklearn.linear_model import LogisticRegressionCV

In [3]:
net_type = 'resnet'
print('evaluate the LID estimator')
score_list = ['LID_10', 'LID_20', 'LID_30', 'LID_40', 'LID_50', 'LID_60', 'LID_70', 'LID_80', 'LID_90']
dataset = 'cifar10'
outf = './adv_output2/' + net_type + '_' + dataset + '/'
out = 'FGSM'

evaluate the LID estimator


In [8]:
def load_characteristics(file_name):
    """
    Load the calculated scores
    return: data and label of input score
    """
    X, Y = None, None
    
    data = np.load(file_name)
    
    if X is None:
        X = data[:, :-1]
    else:
        X = np.concatenate((X, data[:, :-1]), axis=1)
    if Y is None:
        Y = data[:, -1] # labels only need to load once
         
    return X, Y

# LID

## Unsupervised - hyperparameters grid search

In [30]:
for score in score_list:
    for i in range(4):
        X_FGSM, y_FGSM = load_characteristics('./adv_output2/resnet_cifar10/%s_FGSM_data.npy' %score)
        FGSM_auc = roc_auc_score(y_FGSM,X_FGSM[:,i])

        X_PGD, y_PGD = load_characteristics('./adv_output2/resnet_cifar10/%s_PGD_data.npy' %score)
        PGD_auc = roc_auc_score(y_PGD,X_PGD[:,i])

        X_CW, y_CW = load_characteristics('./adv_output2/resnet_cifar10/%s_CW_data.npy' %score)
        CW_auc = roc_auc_score(y_CW,X_CW[:,i])

        print('%s - %d - FGSM auc:%0.4f, PGD auc:%0.4f, CW auc:%0.4f' 
              %(score, i, FGSM_auc, PGD_auc, CW_auc))

LID_10 - 0 - FGSM auc:0.8137, PGD auc:0.8033, CW auc:0.6574
LID_10 - 1 - FGSM auc:0.9997, PGD auc:0.9645, CW auc:0.6523
LID_10 - 2 - FGSM auc:0.9984, PGD auc:0.9040, CW auc:0.6371
LID_10 - 3 - FGSM auc:0.9492, PGD auc:0.8838, CW auc:0.7406
LID_20 - 0 - FGSM auc:0.7543, PGD auc:0.7453, CW auc:0.6122
LID_20 - 1 - FGSM auc:1.0000, PGD auc:0.9876, CW auc:0.6179
LID_20 - 2 - FGSM auc:0.9999, PGD auc:0.9458, CW auc:0.6047
LID_20 - 3 - FGSM auc:0.9808, PGD auc:0.9287, CW auc:0.7343
LID_30 - 0 - FGSM auc:0.7080, PGD auc:0.7030, CW auc:0.5832
LID_30 - 1 - FGSM auc:1.0000, PGD auc:0.9924, CW auc:0.6022
LID_30 - 2 - FGSM auc:1.0000, PGD auc:0.9722, CW auc:0.5780
LID_30 - 3 - FGSM auc:0.9933, PGD auc:0.9457, CW auc:0.6855
LID_40 - 0 - FGSM auc:0.6670, PGD auc:0.6663, CW auc:0.5657
LID_40 - 1 - FGSM auc:1.0000, PGD auc:0.9941, CW auc:0.5798
LID_40 - 2 - FGSM auc:0.9999, PGD auc:0.9777, CW auc:0.5737
LID_40 - 3 - FGSM auc:0.9958, PGD auc:0.9484, CW auc:0.6281
LID_50 - 0 - FGSM auc:0.6401, PGD auc:0.

## Supervised - Hyperparameters grid search

In [19]:
for score in score_list:
    total_X, total_Y = lib_regression.load_characteristics(score, dataset, out, outf)
    X_val, Y_val, X_test, Y_test = lib_regression.block_split_adv(total_X, total_Y)
    pivot = int(X_val.shape[0] / 6)
    X_train = np.concatenate((X_val[:pivot], X_val[2*pivot:3*pivot], X_val[4*pivot:5*pivot]))
    Y_train = np.concatenate((Y_val[:pivot], Y_val[2*pivot:3*pivot], Y_val[4*pivot:5*pivot]))
    X_val_for_test = np.concatenate((X_val[pivot:2*pivot], X_val[3*pivot:4*pivot], X_val[5*pivot:]))
    Y_val_for_test = np.concatenate((Y_val[pivot:2*pivot], Y_val[3*pivot:4*pivot], Y_val[5*pivot:]))
    lr = LogisticRegressionCV(n_jobs=-1).fit(X_train, Y_train)
    
    y_pred = lr.predict_proba(X_val_for_test)[:, 1]
    results = lib_regression.detection_performance(lr, X_val_for_test, Y_val_for_test, outf)
    
    X_FGSM, y_FGSM = load_characteristics('./adv_output2/resnet_cifar10/%s_FGSM_data.npy' %score)
    results_FGSM = lib_regression.detection_performance(lr, X_FGSM, y_FGSM, outf)
    
    X_PGD, y_PGD = load_characteristics('./adv_output2/resnet_cifar10/%s_PGD_data.npy' %score)
    results_PGD = lib_regression.detection_performance(lr, X_PGD, y_PGD, outf)
    
    X_CW, y_CW = load_characteristics('./adv_output2/resnet_cifar10/%s_CW_data.npy' %score)
    results_CW = lib_regression.detection_performance(lr, X_CW, y_CW, outf)
    print('%s - Original auc:%0.4f, FGSM auc:%0.4f, PGD auc:%0.4f, CW auc:%0.4f' 
          %(score, results['TMP']['AUROC'], results_FGSM['TMP']['AUROC'], results_PGD['TMP']['AUROC'], 
            results_CW['TMP']['AUROC']))

LID_10 - Original auc:0.9994, FGSM auc:1.0000, PGD auc:0.9728, CW auc:0.7112
LID_20 - Original auc:1.0000, FGSM auc:1.0000, PGD auc:0.9882, CW auc:0.6472
LID_30 - Original auc:1.0000, FGSM auc:1.0000, PGD auc:0.9952, CW auc:0.6473
LID_40 - Original auc:1.0000, FGSM auc:1.0000, PGD auc:0.9950, CW auc:0.6197
LID_50 - Original auc:1.0000, FGSM auc:1.0000, PGD auc:0.9949, CW auc:0.5914
LID_60 - Original auc:1.0000, FGSM auc:1.0000, PGD auc:0.9941, CW auc:0.5654
LID_70 - Original auc:1.0000, FGSM auc:1.0000, PGD auc:0.9933, CW auc:0.5405
LID_80 - Original auc:1.0000, FGSM auc:1.0000, PGD auc:0.9913, CW auc:0.5152
LID_90 - Original auc:1.0000, FGSM auc:1.0000, PGD auc:0.9888, CW auc:0.5020


# Mahalanobis

In [33]:
print('evaluate the Mahalanobis estimator')
mahalanbis_score_list = ['Mahalanobis_0.0', 'Mahalanobis_0.01', 'Mahalanobis_0.005', \
              'Mahalanobis_0.002', 'Mahalanobis_0.0014', 'Mahalanobis_0.001', 'Mahalanobis_0.0005']

evaluate the Mahalanobis estimator


In [34]:
for score in mahalanbis_score_list:
    for i in range(4):
        X_FGSM, y_FGSM = load_characteristics('./adv_output2/resnet_cifar10/%s_FGSM_data.npy' %score)
        FGSM_auc = roc_auc_score(y_FGSM,X_FGSM[:,i])

        X_PGD, y_PGD = load_characteristics('./adv_output2/resnet_cifar10/%s_PGD_data.npy' %score)
        PGD_auc = roc_auc_score(y_PGD,X_PGD[:,i])

        X_CW, y_CW = load_characteristics('./adv_output2/resnet_cifar10/%s_CW_data.npy' %score)
        CW_auc = roc_auc_score(y_CW,X_CW[:,i])

        print('%s - %d - FGSM auc:%0.4f, PGD auc:%0.4f, CW auc:%0.4f' 
              %(score, i, FGSM_auc, PGD_auc, CW_auc))

Mahalanobis_0.0 - 0 - FGSM auc:0.0989, PGD auc:0.1723, CW auc:0.5265
Mahalanobis_0.0 - 1 - FGSM auc:0.0000, PGD auc:0.0016, CW auc:0.5381
Mahalanobis_0.0 - 2 - FGSM auc:0.0000, PGD auc:0.0003, CW auc:0.5678
Mahalanobis_0.0 - 3 - FGSM auc:0.0000, PGD auc:0.0000, CW auc:0.6139
Mahalanobis_0.01 - 0 - FGSM auc:0.1643, PGD auc:0.4384, CW auc:0.5278
Mahalanobis_0.01 - 1 - FGSM auc:0.0000, PGD auc:0.7450, CW auc:0.4658
Mahalanobis_0.01 - 2 - FGSM auc:0.0012, PGD auc:0.5182, CW auc:0.4655
Mahalanobis_0.01 - 3 - FGSM auc:0.0000, PGD auc:0.3064, CW auc:0.4921
Mahalanobis_0.005 - 0 - FGSM auc:0.1404, PGD auc:0.2738, CW auc:0.5295
Mahalanobis_0.005 - 1 - FGSM auc:0.0000, PGD auc:0.0338, CW auc:0.4752
Mahalanobis_0.005 - 2 - FGSM auc:0.0000, PGD auc:0.2006, CW auc:0.4694
Mahalanobis_0.005 - 3 - FGSM auc:0.0000, PGD auc:0.0296, CW auc:0.5179
Mahalanobis_0.002 - 0 - FGSM auc:0.1192, PGD auc:0.1949, CW auc:0.5270
Mahalanobis_0.002 - 1 - FGSM auc:0.0000, PGD auc:0.0024, CW auc:0.5023
Mahalanobis_0.002 

In [35]:
for score in mahalanbis_score_list:
    total_X, total_Y = lib_regression.load_characteristics(score, dataset, out, outf)
    X_val, Y_val, X_test, Y_test = lib_regression.block_split_adv(total_X, total_Y)
    pivot = int(X_val.shape[0] / 6)
    X_train = np.concatenate((X_val[:pivot], X_val[2*pivot:3*pivot], X_val[4*pivot:5*pivot]))
    Y_train = np.concatenate((Y_val[:pivot], Y_val[2*pivot:3*pivot], Y_val[4*pivot:5*pivot]))
    X_val_for_test = np.concatenate((X_val[pivot:2*pivot], X_val[3*pivot:4*pivot], X_val[5*pivot:]))
    Y_val_for_test = np.concatenate((Y_val[pivot:2*pivot], Y_val[3*pivot:4*pivot], Y_val[5*pivot:]))
    lr = LogisticRegressionCV(n_jobs=-1).fit(X_train, Y_train)
    
    y_pred = lr.predict_proba(X_val_for_test)[:, 1]
    results = lib_regression.detection_performance(lr, X_val_for_test, Y_val_for_test, outf)
    
    X_FGSM, y_FGSM = load_characteristics('./adv_output2/resnet_cifar10/%s_FGSM_data.npy' %score)
    results_FGSM = lib_regression.detection_performance(lr, X_FGSM, y_FGSM, outf)
    
    X_PGD, y_PGD = load_characteristics('./adv_output2/resnet_cifar10/%s_PGD_data.npy' %score)
    results_PGD = lib_regression.detection_performance(lr, X_PGD, y_PGD, outf)
    
    X_CW, y_CW = load_characteristics('./adv_output2/resnet_cifar10/%s_CW_data.npy' %score)
    results_CW = lib_regression.detection_performance(lr, X_CW, y_CW, outf)
    print('%s - Original auc:%0.4f, FGSM auc:%0.4f, PGD auc:%0.4f, CW auc:%0.4f' 
          %(score, results['TMP']['AUROC'], results_FGSM['TMP']['AUROC'], results_PGD['TMP']['AUROC'], 
            results_CW['TMP']['AUROC']))

Mahalanobis_0.0 - Original auc:1.0000, FGSM auc:1.0000, PGD auc:0.9991, CW auc:0.4257
Mahalanobis_0.01 - Original auc:1.0000, FGSM auc:0.9996, PGD auc:0.3380, CW auc:0.5330
Mahalanobis_0.005 - Original auc:1.0000, FGSM auc:1.0000, PGD auc:0.9025, CW auc:0.5276
Mahalanobis_0.002 - Original auc:1.0000, FGSM auc:1.0000, PGD auc:0.9953, CW auc:0.5021
Mahalanobis_0.0014 - Original auc:1.0000, FGSM auc:1.0000, PGD auc:0.9980, CW auc:0.4841
Mahalanobis_0.001 - Original auc:1.0000, FGSM auc:1.0000, PGD auc:0.9987, CW auc:0.4657
Mahalanobis_0.0005 - Original auc:1.0000, FGSM auc:1.0000, PGD auc:0.9991, CW auc:0.4383


In [19]:
out = 'FGSM'
best_auroc, best_result, best_index = 0, 0, 0
score = score_list[5]
print('load train data: ', out, ' of ', score)
total_X, total_Y = lib_regression.load_characteristics(score, dataset, out, outf)
X_val, Y_val, X_test, Y_test = lib_regression.block_split_adv(total_X, total_Y)
pivot = int(X_val.shape[0] / 6)
X_train = np.concatenate((X_val[:pivot], X_val[2*pivot:3*pivot], X_val[4*pivot:5*pivot]))
Y_train = np.concatenate((Y_val[:pivot], Y_val[2*pivot:3*pivot], Y_val[4*pivot:5*pivot]))
X_val_for_test = np.concatenate((X_val[pivot:2*pivot], X_val[3*pivot:4*pivot], X_val[5*pivot:]))
Y_val_for_test = np.concatenate((Y_val[pivot:2*pivot], Y_val[3*pivot:4*pivot], Y_val[5*pivot:]))
lr = LogisticRegressionCV(n_jobs=-1).fit(X_train, Y_train)
y_pred = lr.predict_proba(X_train)[:, 1]
#print('training mse: {:.4f}'.format(np.mean(y_pred - Y_train)))
y_pred = lr.predict_proba(X_val_for_test)[:, 1]
#print('test mse: {:.4f}'.format(np.mean(y_pred - Y_val_for_test)))
results = lib_regression.detection_performance(lr, X_val_for_test, Y_val_for_test, outf)
if best_auroc < results['TMP']['AUROC']:
    best_auroc = results['TMP']['AUROC']
    best_index = score
    best_result = lib_regression.detection_performance(lr, X_test, Y_test, outf)
list_best_results_out.append(best_result)
list_best_results_index_out.append(best_index)

load train data:  FGSM  of  Mahalanobis_0.001


In [20]:
X_val_for_test.shape

(420, 4)

In [21]:
list_best_results_out

[{'TMP': {'AUIN': 0.99990079365079365,
   'AUOUT': 0.9998015873015873,
   'AUROC': 1.0,
   'DTACC': 1.0,
   'TNR': 1.0}}]

In [22]:
y_pred = lr.predict_proba(X_CW)[:, 1]
print('test mse: {:.4f}'.format(np.mean(y_pred - y_CW)))
results = lib_regression.detection_performance(lr, X_CW, y_CW, outf)
results

test mse: -0.5000


{'TMP': {'AUIN': 0.37303626869953449,
  'AUOUT': 0.37071534775087356,
  'AUROC': 0.28474323979591837,
  'DTACC': 0.5,
  'TNR': 0.0053571428571428381}}

In [ ]:

count_in = 0
print("results of Mahalanobis")
for in_list in list_best_results_ours:
    print('in_distribution: ' + dataset_list[count_in] + '==========')
    count_out = 0
    for results in in_list:
        print('out_distribution: '+ adv_test_list[count_out])
        for mtype in mtypes:
            print(' {mtype:6s}'.format(mtype=mtype), end='')
        print('\n{val:6.2f}'.format(val=100.*results['TMP']['TNR']), end='')
        print(' {val:6.2f}'.format(val=100.*results['TMP']['AUROC']), end='')
        print(' {val:6.2f}'.format(val=100.*results['TMP']['DTACC']), end='')
        print(' {val:6.2f}'.format(val=100.*results['TMP']['AUIN']), end='')
        print(' {val:6.2f}\n'.format(val=100.*results['TMP']['AUOUT']), end='')
        print('Input noise: ' + list_best_results_index_ours[count_in][count_out])
        print('')
        count_out += 1
    count_in += 1

In [ ]:
# for dataset in dataset_list:
#     print('load train data: ', dataset)
#     outf = './adv_output/' + args.net_type + '_' + dataset + '/'

#     list_best_results_out, list_best_results_index_out = [], []
#     for out in adv_test_list:
#         best_auroc, best_result, best_index = 0, 0, 0
#         for score in score_list:
#             print('load train data: ', out, ' of ', score)
#             total_X, total_Y = lib_regression.load_characteristics(score, dataset, out, outf)
#             X_val, Y_val, X_test, Y_test = lib_regression.block_split_adv(total_X, total_Y)
#             pivot = int(X_val.shape[0] / 6)
#             X_train = np.concatenate((X_val[:pivot], X_val[2*pivot:3*pivot], X_val[4*pivot:5*pivot]))
#             Y_train = np.concatenate((Y_val[:pivot], Y_val[2*pivot:3*pivot], Y_val[4*pivot:5*pivot]))
#             X_val_for_test = np.concatenate((X_val[pivot:2*pivot], X_val[3*pivot:4*pivot], X_val[5*pivot:]))
#             Y_val_for_test = np.concatenate((Y_val[pivot:2*pivot], Y_val[3*pivot:4*pivot], Y_val[5*pivot:]))
#             lr = LogisticRegressionCV(n_jobs=-1).fit(X_train, Y_train)
#             y_pred = lr.predict_proba(X_train)[:, 1]
#             #print('training mse: {:.4f}'.format(np.mean(y_pred - Y_train)))
#             y_pred = lr.predict_proba(X_val_for_test)[:, 1]
#             #print('test mse: {:.4f}'.format(np.mean(y_pred - Y_val_for_test)))
#             results = lib_regression.detection_performance(lr, X_val_for_test, Y_val_for_test, outf)
#             if best_auroc < results['TMP']['AUROC']:
#                 best_auroc = results['TMP']['AUROC']
#                 best_index = score
#                 best_result = lib_regression.detection_performance(lr, X_test, Y_test, outf)
#         list_best_results_out.append(best_result)
#         list_best_results_index_out.append(best_index)
#     list_best_results.append(list_best_results_out)
#     list_best_results_index.append(list_best_results_index_out)